# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-30.33,90,0,1.03,CA,1733304977
1,1,iqaluit,63.7506,-68.5145,-16.15,72,100,4.12,CA,1733305181
2,2,san cristobal de las casas,16.7500,-92.6333,7.53,85,98,3.43,MX,1733305183
3,3,adamstown,-25.0660,-130.1015,23.22,82,100,6.58,PN,1733305184
4,4,daru,-9.0763,143.2092,27.83,81,97,4.78,PG,1733305185


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",           
    y="Lat",            
    size="Humidity",         
    geo=True,               
    tiles="OSM",            
    color="City",            
    width = 1000,
    height = 600,
    alpha=0.6,              
    hover_cols=["City", "Country", "Humidity"] 
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#Filtering data based on the Max Temp between 23 and 30
filtered_data = city_data_df[(city_data_df["Max Temp"]>23)&(city_data_df["Max Temp"]<30) ]

# Drop any rows with null values
filtered_data = filtered_data.dropna()

# Display sample data
filtered_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,adamstown,-25.0660,-130.1015,23.22,82,100,6.58,PN,1733305184
4,4,daru,-9.0763,143.2092,27.83,81,97,4.78,PG,1733305185
8,8,howard springs,-12.4970,131.0470,28.95,82,31,3.16,AU,1733305189
25,25,kayangel,8.0823,134.7173,28.88,76,100,3.63,PW,1733305207
28,28,port alfred,-33.5906,26.8910,23.99,74,72,2.24,ZA,1733305211
...,...,...,...,...,...,...,...,...,...,...
553,553,abu dhabi,24.4667,54.3667,27.06,32,0,3.60,AE,1733305812
554,554,sirkazhi,11.2333,79.7333,29.37,69,90,2.86,IN,1733305813
558,558,port-gentil,-0.7193,8.7815,24.98,94,75,2.57,GA,1733305818
561,561,mili,6.0815,171.7350,28.25,78,99,7.56,MH,1733305821


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_data[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,82,
4,daru,PG,-9.0763,143.2092,81,
8,howard springs,AU,-12.4970,131.0470,82,
25,kayangel,PW,8.0823,134.7173,76,
28,port alfred,ZA,-33.5906,26.8910,74,
...,...,...,...,...,...,...
553,abu dhabi,AE,24.4667,54.3667,32,
554,sirkazhi,IN,11.2333,79.7333,69,
558,port-gentil,GA,-0.7193,8.7815,94,
561,mili,MH,6.0815,171.7350,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius =10000
params = {
    "categories" : "accommodation.hotel",
    "limit" : 1,
    "apiKey" : geoapify_key,
    "radius" : radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    pprint(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'features': [], 'type': 'FeatureCollection'}
adamstown - nearest hotel: No hotel found
{'features': [{'geometry': {'coordinates': [143.21163989999997,
                                            -9.07362359983593],
                            'type': 'Point'},
               'properties': {'address_line1': 'Kuki',
                              'address_line2': 'Murray (Mangopari) Road, Daru '
                                               'Western, Papua New Guinea',
                              'categories': ['accommodation',
                                             'accommodation.hotel'],
                              'city': 'Daru',
                              'country': 'Papua New Guinea',
                              'country_code': 'pg',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Ope

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,82,No hotel found
4,daru,PG,-9.0763,143.2092,81,Kuki
8,howard springs,AU,-12.4970,131.0470,82,Rydges Palmerston
25,kayangel,PW,8.0823,134.7173,76,No hotel found
28,port alfred,ZA,-33.5906,26.8910,74,No hotel found
...,...,...,...,...,...,...
553,abu dhabi,AE,24.4667,54.3667,32,Marriott Downtown Abu-Dhabi
554,sirkazhi,IN,11.2333,79.7333,69,No hotel found
558,port-gentil,GA,-0.7193,8.7815,94,Tara-Mé
561,mili,MH,6.0815,171.7350,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",           
    y="Lat",           
    size="Humidity",        
    geo=True,                
    tiles="OSM",             
    color="City",            
    alpha=0.6,  
    width = 1000,             
    height = 600,
    hover_cols=["City", "Country", "Humidity","Hotel Name"]  # Info to display on hover
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)